In [ ]:
# Load all dependencies
using Plots
using SpecialFunctions, LinearAlgebra
using LaTeXStrings
using Dates
using Memoize

using Images

using CSV

using Statistics

In [ ]:
# Auxiliary functions

# default figure style
res = 640
function newfig()
    Figure(size = (res,res), fontsize = 24)
end

# file name for temporary figures
function fname()
    tt = replace(string(now()),":"=>"-")
    tt = replace(tt,"."=>"-")
    ext = ".png"
    path = "c:\\Users\\Dorin\\Dropbox\\Julia\\figures\\"
    join([path,tt,ext])
end

# mimic Mathematica's chop function
# adjust tolerance to adequate value to avoid evaluations on unnecessary tiny numbers
function chop(x::Real; tol=1e-20)
    abs(x) < tol ? zero(x) : x
end

function chop(z::Complex; tol=1e-20)
    real_part = abs(real(z)) < tol ? 0.0 : real(z)
    imag_part = abs(imag(z)) < tol ? 0.0 : imag(z)
    complex(real_part, imag_part)
end

# A matrix of values V(ϕ,l) = exp(-i l (ϕ-π/2)), used for fast evaluation of the scattering amplitude
function fmat(xs)
    [exp(-im*l*(x-pi/2)) for x in xs, l in ls]
end

# A matrix of values V(ϕ,l) = exp(-i l (ϕ-π/2)), used for fast evaluation of the scattering amplitude
function fmat(xs,lmax)
    [exp(-im*l*(x-pi/2)) for x in xs, l in -lmax:lmax]
end

# The signed angle between two 2D vectors, used to evaluate αij
function angbetween(v1,v2)
    atan(v1[1]*v2[2]-v1[2]*v2[1],v1[1]*v2[1]+v1[2]*v2[2])
end

# Spacing ratios
function rd(δ)
    n = length(δ)
    return δ[2:n]./δ[1:n-1]
end

# Definition of r-tilde = min(r,1/r)
function rx(x)
    return min(x,1/x)
end

# GOE distribution of spacings
function pdfO(x)
    pi/2 * x * exp(-pi/4*x^2)
end

# GOE distribution of r-tilde
function pdfOR(r)
    54/8 * (r+r^2) / (1+r+r^2)^(5/2)
end




In [ ]:
# Shorthand for Bessel and Hankel functions, memoized to avoid repeated evaluation
@memoize h1(n,x) = hankelh1(n,x)
@memoize j(n,x) = besselj(n,x)

In [ ]:
# Define the system: 3 or 2 disk (equilateral triangle, changing sizes)

# Placement of the disks
L = 3.
d1 = [L/sqrt(3), 0]
d2 = [-L/sqrt(3)/2, L/2]
d3 = [-L/sqrt(3)/2, -L/2]
d0 = [0,0];

da = [d1, d2, d3]

# Sizes of the disks
eps = 0.2
R1 = 1.
R2 = 1+eps
R3 = 1-eps
Ra = [R1, R2, R3]


# Number of disks, set to 2 or 3
# if nd = 2 keep only the disks R1 and R2
nd = 3;

In [ ]:
# Define the system: 4-disks

# Placement of the disks
L = 4.
d1 = [L/2,0]
d2 = [-L/2,0]
d3 = [0, L/2]
d4 = [0,-L/2]
d0 = [0,0];

da = [d1, d2, d3, d4]

# Sizes of the disks
eps = 0.
R1 = 1.2
R2 = 1.1
R3 = 1.0
R4 = 0.9

Ra = [R1, R2, R3, R4]


# Set number of disks
nd = 4;

# In principle the code below will run for any nd,
# but I've not implemented any way to define a system automatically,
# so one should define the positions and sizes manually

In [ ]:
#Define all the variables that enter the formulas in the computation of the S-matrix
# Don't forget to run after changing configuration!
sa = [norm(da[a]-d0) for a in 1:nd]
Lab = [norm(da[a]-da[b]) for a in 1:nd, b in 1:nd]
chia = [atan(da[a][2],da[a][1]) for a in 1:nd]
aab = [angbetween(da[a],da[b]-da[a])*(1-(a==b)) for a = 1:nd, b = 1:nd]

In [ ]:
# Components of the C, D, and M-matrices
# Formulas from Wirzba 1997
function Cl(k,l,a)
    [2 * im / pi / Ra[a] * exp(im*l*chia[a]) * j(l-m,k*sa[a]) / h1(m,k*Ra[a]) for m in ms]
end

function Cl(k,l)
    reduce(vcat,[Cl(k,l,a) for a in 1:nd])
end

function Dl(k,l,a)
    [pi * Ra[a] * j(l-m,k*sa[a]) * j(m,k*Ra[a]) * exp(-im*l*chia[a]) for m in ms]
end

function Dl(k,l)
    reduce(vcat,[Dl(k,l,a) for a in 1:nd])
end

function MM(k,a,b)
    if a == b
        return [ComplexF64(1)*(m1==m2) for m1 in ms, m2 in ms]
    else
        return [Ra[a] / Ra[b] * j(m1,k*Ra[a]) / h1(m2,k*Ra[b]) * h1(m1-m2,k*Lab[a,b]) * exp(im*(m1 * aab[a,b]-m2*(aab[b,a]-pi))) for m1 in ms, m2 in ms]
    end
end

function MM(k)
    res = Array{Matrix{ComplexF64}}(undef,nd^2)
    for a = 1:nd
        for b = 1:nd
            res[nd*(a-1)+b] = MM(k,a,b)
        end
    end
    hvcat(nd,res...)
end    

In [ ]:
# Compute the M-matrix

# Set value of k here
k = 25

# Cut-off size of the M-matrices:
# We set it to an empirical value of 1.7k,
# but this assumes that the lengths in the system are of order 1, adjust if necessary
mu = max(round(1.7*k),20)
ms = -mu:mu

bigM = chop.(MM(k))
iM = @time chop.(inv(bigM));

# Draw heatmap of M to verify that elements outside the mu*mu blocks are zero,
# as a basic consistency check.
heatmap(abs.(bigM))

In [ ]:
# Cut-off value for the S-matrix size: set empirically to nd*mu
lmax = Int64(2*nd*mu)
ls = -lmax:lmax

# T = @time [transpose(Cl(k,l1)) * iM * Dl(k,l2) for l1 in ls, l2 in ls]
is = Int32.(ls.+lmax.+1)
@time Cls = [chop.(Cl(k,l1)) for l1 in ls]
@time Dls = [chop.(Dl(k,l2)) for l2 in ls]

T = Matrix{ComplexF64}(undef,2*lmax+1,2*lmax+1)
@time for i in is
    CiM = transpose(Cls[i]) * iM
    for j in is
        T[i,j] = CiM*Dls[j]
    end
end
# T = @time [transpose(Cls[i]) * iM * Dls[j] for i in is, j in is]
S = I + im * T;

# Unitarity check, if failed, increase cut-offs and try again
lambdas = eigvals(S)
maxerr = maximum(abs.(abs.(eigvals(S)).-1))
if maxerr >1e-9
    println("UNITARITY CHECK FAILED!")
end
println(maxerr)

In [ ]:
# Two dimensional plots of the cross-section

# Number of points
Np = 2 * res


# Full range
phi1_min = -pi
phi1_max = pi
phi2_min = -pi
phi2_max = 2*pi

# #Zoomed-in range
# phi1_min = -pi+1
# phi1_max = -pi+3
# phi2_min = 0
# phi2_max = 2
# Np = 2 * res

# # Zoomed-in range
# phi1_min = -1.5
# phi1_max = -1.3
# phi2_min = 1.3
# phi2_max = 1.4

# Set aspect ratio to be 1 in the figures
ratio = (phi2_max-phi2_min)/(phi1_max-phi1_min)

phi1 = collect(range(phi1_min,phi1_max,Np))
phi2 = collect(range(phi2_min,phi2_max,Int64(round(ratio*Np))))

fl1 = fmat(phi1)
fl2 = fmat(phi2)

# The amplitude is evaluated by simple matrix multiplication:
# f = f(ϕ1,l1) * T(l1,l2) * f†(l2,ϕ2)
amplitude = @time fl1 * T * adjoint(fl2)

# The differential cross section (with appropriate normalization is):
dsigma = 1/2/pi/k*abs.(amplitude).^2

In [ ]:
# Plot of dσ

# Set the maximum value to be plotted to see details of the plot
# Otherwise the large values on the line ϕ1=ϕ2 will hide all other features
threshold = 10

# heatmap plot
# Note: title is set manually for now, don't forget to change
fig = @time heatmap(phi2,phi1,min.(dsigma,threshold),
    color=reverse(cgrad(:Blues,12,categorical=true)),legend=false,
    aspect_ratio=:equal,
    size=(Int64(round(ratio*res)),res),
    tickfontsize=16,titlefontsize=18,
    title="k = 25 (four-disk)",
    interpolate=false
    )
xlims!(phi2_min,phi2_max)
ylims!(phi1_min,phi1_max)
@time savefig(fname())
@time fig

In [ ]:
# Find local maxima of dsigma
# If resolution is high enough, should be close to the positions of the peaks

ixs= @time Images.findlocalmaxima(dsigma)

ps = []


# Exclude points outside the chosen fundamental region (parity symmetry)
for ix in Tuple.(ixs)
    p = [phi1[ix[1]],phi2[ix[2]]]
    if (p[1] < p[2] - 0.1 && p[1] >=  p[2] - pi + 0.1)
        push!(ps,p)
    end
end

# Display total number of points, total number of peaks, and number of peaks in the region
[length(phi1)*length(phi2),length(ixs),length(ps)]

In [ ]:
# Find the local maxima on a discretized grid introduces some false positives
# Eliminate points that are too near to each other by setting a threshold (manually at this stage)

# Filter peaks: throw eigenvalues that are too near another one
peaks = []
c1 = []
c2 = []
threshold = 0.020
push!(peaks,ps[1])
for i in 2:length(ps)
    p = ps[i]
    (mm, mpos) = findmin([norm(p-peak) for peak in peaks])
    if mm > threshold
        push!(peaks,p)
        push!(c1,p[1])
        push!(c2,p[2])
    end
end

δij = []
for i in 1:length(ps)-1
    for j in (i+1):length(peaks)
        mm = norm(peaks[i]-peaks[j])
        if mm < 0.05
            push!(δij,norm(peaks[i]-peaks[j]))
        end
    end
end

# Check that histogram of spacings now doesn't have the maximum near zero
# (which was always an artifact of the way peaks are found)
histogram(δij)
# sort(δij)[1:150]

In [ ]:
# Mar locations of peaks in plot of function
plot(fig)
scatter!(c2,c1,markersize=2.5,aspectratio=:equal,color=:yellow)
plot!([-pi,pi,2*pi,0,-pi],[-pi,pi,pi,-pi,-pi],color=:red,linewidth=2)
savefig(fname())

In [ ]:
# Save locations of peaks to file
CSV.write("lambda_k50_twodisk.csv",(x = c1, y = c2))

In [ ]:
# Plot distribution of spacings of eigenvalues of the S-matrix
ϕn = sort(mod.(angle.(lambdas),2*pi))
ϕn = filter(x -> (2*pi - x > 0.1) & (x > 0.1), ϕn)
δn = diff(ϕn)
δn = δn / mean(δn)

histogram(δn, normalize=:pdf, bins=100, alpha=0.5,label=false)

x = collect(range(0,3,1000))
plot!(x,pdfO.(x),label="GOE")

In [ ]:
# Plot distribution of spacing ratios of eigenvalues of the S-matrix
rn = rx.(rd(δn))
show([mean(rn),4-2*sqrt(3)])

histogram(rn, normalize=:pdf, bins=40, alpha=0.5,aspectratio=1/4,label=false)
xlims!(0,1)
ylims!(0,2)

x = collect(range(0,1,400))
plot!(x,pdfOR.(x),label="GOE")

In [ ]:
# Find a resonance by manual scan near a candidate point
# I haven't implemented any automated way to scan for resonances

# This is near a resonance for the R_i = (1,1.2,0.8), L = 3 system.
x0 = 12.3364;
y0 = -0.3269;

# # 
# x0 = 10.52;
# y0 = -0.3;

kres = x0 + im * y0
dx = 0.0001;
dy = 0.0001;
detM = @time [log(abs(det(MM(x0 + dx * x + im* (y0 - dy * y))))) for x in -3:3, y in -3:3]
xy = @time [[x0+dx*x + im*(y0 - dy*y)] for x in -3:3, y in -3:3]

# After computing detM, see where to move the point to get the minimum...
detM

In [ ]:
# Compute cross-section as a function of k
ks = collect(5:0.05:30)

f1 = []
f2 = []
f3 = []

@time for k in ks
    mu = max(round(1.7*k),20)
    ms = -mu:mu

    bigM = chop.(MM(k))
    iM = chop.(inv(bigM));

    # Cut-off value for the S-matrix size: set empirically to nd*mu
    lmax = Int64(2*nd*mu)
    ls = -lmax:lmax

    
    is = Int32.(ls.+lmax.+1)
    Cls = [chop.(Cl(k,l1)) for l1 in ls]
    Dls = [chop.(Dl(k,l2)) for l2 in ls]

    T = Matrix{ComplexF64}(undef,2*lmax+1,2*lmax+1)
    for i in is
        CiM = transpose(Cls[i]) * iM
        for j in is
            T[i,j] = CiM*Dls[j]
        end
    end
    TTd = T * adjoint(T)
    
    push!(f1,real(1/k * tr(TTd)))
    push!(f2,real(1/k * transpose(fmat(0)) * TTd * conj(fmat(0))))
    push!(f3,real(1/k * transpose(fmat(0.4)) * TTd * conj(fmat(0.4))))
    print("*")
end

In [ ]:
# Plot the results of above cell
plot(ks,f1,
    color=:black,linewidth=2,
    size=(720,480),
    tickfontsize=14,titlefontsize=16,legendfontsize=12,labelfontsize=16,
    label="σ̄",
    title="Symmetric setup",
    xlabel = "k", ylabel = "σ"
)
plot!(ks,real.(f2),color=:blue,linestyle=:dash,label="σ(ϕ=0)")
plot!(ks,real.(f3),color=:red,linestyle=:dash,label="σ(ϕ=0.4)")
savefig(fname())

In [ ]:
# One dimensional plots of the cross-section

# Number of points
Np = 10000


# Full range
phi2_min = -pi
phi2_max = pi

phi2 = collect(range(phi2_min,phi2_max,Np))

fl1 = transpose(fmat(0))
fl2 = fmat(phi2)

# The amplitude is evaluated by simple matrix multiplication:
# f = f(ϕ1,l1) * T(l1,l2) * f†(l2,ϕ2)
amplitude = @time vec(fl1 * T * adjoint(fl2))

# The differential cross section (with appropriate normalization is):
dsigma = 1/2/pi/k*abs.(amplitude).^2

peaks1 = []
for i in 2:Np-1
    if dsigma[i] > dsigma[i+1] && dsigma[i] > dsigma[i-1]
        push!(peaks1,phi2[i])
    end
end

plot(phi2,dsigma,linewidth=1,label=false,
    size=(720,480),
    title = "Asymmetric cofiguration, ϕ=0",
    tickfontsize=14,titlefontsize=16,legendfontsize=12,labelfontsize=16,grid=false)
vline!(peaks1,color=:grey,opacity=0.2,label=false)
xlabel!(L"\phi\prime")
ylabel!(L"|f(k;\phi,\phi\prime)|^2")
ylims!(0,7)
xlims!(-pi,pi)
savefig(fname())